In [1]:
import os
import json
import re

regx_one = re.compile('.*?(\\d+)', re.IGNORECASE|re.DOTALL)
regx_two = re.compile('.*?(\\d+).*?(\\d+)', re.IGNORECASE|re.DOTALL)

In [2]:
def get_path_data(txt: str) -> str:
    m = regx_two.search(txt)
    if m:
        int1=m.group(1)
        int2=m.group(2)
        return int1, int2

In [3]:
def read_json(file_name: str):
    with open(file_name) as json_file:
        data = json.load(json_file)
    
    return data

In [4]:
if "requirements.txt" not in os.listdir():
    os.chdir("../")

In [5]:
root = "./resources"

score_cards, summaries, overviews = [], [], []

for dirname, dirs, files in os.walk(root):
    if "scorecards" in dirname:
        score_cards.append(dirname)
    
    if "summary.json" in files:
        summaries.append("{}/{}".format(dirname, "summary.json"))
    
    if "overview.json" in files:
        overviews.append("{}/{}".format(dirname, "overview.json"))

In [9]:
print("Number of Summary files to process: {}".format(len(summaries)))
player_storage = []

for each_summary_file in summaries:
    summary_1 = read_json(each_summary_file)
    for each_player in summary_1["leaderboard"]["players"]:
        bio = each_player['player_bio']
        bio["player_id"] = each_player["player_id"]
        player_storage.append(bio)
        
print("Number of Players: {}".format(len(player_storage)))

Number of Summary files to process: 175
Number of Players: 22363


In [7]:
shot_storage = []
card_counter = 0

for each_score_card in score_cards:
    for each_card in os.listdir(each_score_card):
        card_counter += 1
        
        score_cards_1 = read_json("{}/{}".format(each_score_card, each_card))
        (tournament_id, year), player_id = get_path_data(each_score_card), int(score_cards_1["p"]["id"])
        for rnd_inx, round_ in enumerate(score_cards_1["p"]["rnds"]):
            for hole_inx, hole_ in enumerate(round_["holes"]):
                for each_shot in hole_["shots"]:
                    each_shot["tournament_id"] = tournament_id
                    each_shot["year"] = year
                    each_shot["player_id"] = player_id
                    each_shot["round"] = rnd_inx + 1
                    each_shot["hole"] = hole_inx + 1
                    shot_storage.append(each_shot)
    
    print("Cards handled: {}".format(card_counter))
                    
print("{} cards processed totalling {} shots".format(card_counter, len(shot_storage)))

Cards handled: 93
Cards handled: 192
Cards handled: 287
Cards handled: 383
Cards handled: 503
Cards handled: 623
Cards handled: 743
Cards handled: 863
Cards handled: 881
Cards handled: 899
Cards handled: 917
Cards handled: 1037
Cards handled: 1157
Cards handled: 1277
Cards handled: 1397
Cards handled: 1472
Cards handled: 1552
Cards handled: 1627
Cards handled: 1688
Cards handled: 1820
Cards handled: 1952
Cards handled: 2084
Cards handled: 2216
Cards handled: 2360
Cards handled: 2504
Cards handled: 2660
Cards handled: 2816
Cards handled: 2972
Cards handled: 3128
Cards handled: 3284
Cards handled: 3440
Cards handled: 3571
Cards handled: 3702
Cards handled: 3834
Cards handled: 3966
Cards handled: 4063
Cards handled: 4160
Cards handled: 4253
Cards handled: 4342
Cards handled: 4498
Cards handled: 4654
Cards handled: 4810
Cards handled: 4966
Cards handled: 5110
Cards handled: 5254
Cards handled: 5398
Cards handled: 5530
Cards handled: 5661
Cards handled: 5793
Cards handled: 5937
Cards handle

In [20]:
import pandas as pd

In [31]:
player_storage = pd.DataFrame(player_storage)

In [38]:
player_storage.to_parquet('player_storage.parquet', engine='fastparquet', 
                          compression="gzip", index=False)

In [33]:
os.getcwd()

'/Users/dandixey/Scala/kratos/StrokesGained/data/GameGeneration'

In [34]:
shot_storage = pd.DataFrame(shot_storage)

In [39]:
shot_storage.to_parquet('shot_storage.parquet', engine='fastparquet', 
                        compression='gzip', index=False)

In [44]:
shot_storage.columnsumns

Index(['n', 'pid', 'time', 'putt', 't', 'prv', 'tee', 'cup', 'from', 'to',
       'asc', 'dist', 'left', 'x', 'y', 'z', 'club', 'con', 'shotText',
       'tournament_id', 'year', 'player_id', 'round', 'hole'],
      dtype='object')

In [60]:
shot_storage[(shot_storage["player_id"] == 29289) &
             (shot_storage["tournament_id"] == '505') &
             (shot_storage["year"] == '2014') &
             (shot_storage["hole"] == 1) &
             (shot_storage["round"] == 1)
            ][["n", 'x', 'y', 'z', 'club', 'shotText']]

,n,x,y,z,club,shotText
0,1,8595.358,7456.476,106.198,CFW,"Shot 1 232 yds to right fairway, 125 yds to hole"
1,2,8332.529,7213.823,109.089,CUK,"Shot 2 119 yds to green, 19 ft to hole"
2,3,8322.630,7195.538,109.566,CUK,"Shot 3 putt 20 ft 10 in., 23 in. to hole"
3,4,0,0,0,CUK,Shot 4 in the hole


In [ ]:
distance = math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2 + (z1 - z2) ** 2)